# Libraries

In [2]:
import ast
import math
import os
import gc

import cv2
from pathlib import Path

from PIL import Image, ImageSequence
import numpy as np
from numpy.fft import fft2, fftshift, ifft2, ifftshift
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, argrelextrema
# from skfda import FDataGrid
from numpy import unravel_index
import statistics
import heapq
import pandas as pd

# import tensorflow as tf
# from tensorflow.keras import layers, models

import itertools

from scipy.ndimage import gaussian_filter
from scipy.optimize import curve_fit
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.patches as patches

# Functions

In [2]:
def interpolation(data):
    return(np.interp([i for i in range(1,len(data)*50)],xp=[i*50 for i in range(0,len(data))],fp=data))

In [3]:
def draw_circle(image,ccol,crow,circle_radius):
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    circle_center=(ccol,crow)
    circle = patches.Circle(circle_center, circle_radius, edgecolor='black', facecolor='none', linewidth=1)
    ax.add_patch(circle)

    cross_size = 10
    ax[0].plot([circle_center[0] - cross_size, circle_center[0] + cross_size], 
                [circle_center[1], circle_center[1]], color='white', linewidth=2) 
    ax[0].plot([circle_center[0], circle_center[0]], 
            [circle_center[1] - cross_size, circle_center[1] + cross_size], color='white', linewidth=2) 

    return fig,ax


In [4]:
def process_image_low_movement(filename, output_path, preProcData, result_folder, i,df_list):
    i=i*50
    file_path = os.path.join(output_path, filename)

    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)


    movementNearFramesX=preProcData['diff_center_x'][i:i+100].sum()
    movementNearFramesY=preProcData['diff_center_y'][i:i+100].sum()

    if((movementNearFramesX+movementNearFramesY)>10):
        ccol=preProcData['center_y(ccol)'][i]
        crow=preProcData['center_x(crow)'][i]
    else:
        ccol=preProcData['smooth_center_y(ccol)'][i]
        crow=preProcData['smooth_center_x(crow)'][i]

    circle_radius = preProcData['circle_radius'][i]

    new_data = pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
    })
    df_list.append(new_data)

    fig,ax = draw_circle(image, ccol, crow, circle_radius)
    # fig.imshow(fig, cmap='gray', vmin=0, vmax=255)
    fig.savefig(os.path.join(result_folder, f"result_{filename}"))
    
    if(i%100==0):
        print(i)

    plt.cla()
    plt.close(fig)
    plt.close('all')
    del fig,ax

In [5]:
def process_image_high_movement(filename, output_path, preProcData, result_folder, i,df_list):
    i=i*50
    file_path = os.path.join(output_path, filename)
    
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    ccol=preProcData['center_y(ccol)'][i]
    crow=preProcData['center_x(crow)'][i]
    circle_radius = preProcData['circle_radius'][i]

    new_data = pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
    })
    df_list.append(new_data)

    fig,ax = draw_circle(image, ccol, crow, circle_radius)
    # plt.imshow(fig, cmap='gray', vmin=0, vmax=255)
    fig.savefig(os.path.join(result_folder, f"result_{filename}"))
    
    if(i%100==0):
        print(i)
        
    plt.cla()
    plt.close(fig)
    plt.close('all')
    del fig,ax

<!-- 1 pixel ~ 0.032mikrometrai -->

# Main

In [6]:

def post_processing_mean(test_number,r_in, r_out, ring_index, allowed_change, input_path, input_tiff, output_base_path,output_path,result_folder,excel_input_path,excel_path, interpolationP=True):
    movement_threshold = 3500
    # output_path = Path(output_base_path) / Path(input_tiff).stem
    # print(output_path)
    # output_path.mkdir(parents=True, exist_ok=True)
    test_number = input_tiff.split('.')[1]

    result_folder = f"Results_Validation_Large_Interpolated_Mean/{test_number}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}"
    Path(result_folder).mkdir(parents=True, exist_ok=True)

    preProcData=pd.read_excel(excel_input_path)
    if (interpolationP==True):
        center_x_interpolated=interpolation(preProcData['center_x(crow)'])
        center_y_interpolated=interpolation(preProcData['center_y(ccol)'])
        circle_radius_interpolated=interpolation(preProcData['circle_radius'])
        window=50

        dfList=list(zip(center_x_interpolated,center_y_interpolated,circle_radius_interpolated))
        preProcData=pd.DataFrame(dfList,columns=['center_x(crow)','center_y(ccol)','circle_radius'])
    else:
        window = 3

    # calculating the sum of differences of the center (moving bead or stationary)
    preProcData['diff_center_y'] = preProcData['center_y(ccol)'].diff().abs().fillna(0)
    preProcData['diff_center_x'] = preProcData['center_x(crow)'].diff().abs().fillna(0)
    # preProcData['total_movement'] = preProcData['diff_center_y'] + preProcData['diff_center_x']
    totalMovement=0

    for index, row in preProcData.iterrows():
        totalMovement=totalMovement+row['diff_center_x']+row['diff_center_y']

    preProcData['smooth_center_y(ccol)'] = preProcData['center_y(ccol)'].rolling(window=window).mean()
    preProcData['smooth_center_x(crow)'] = preProcData['center_x(crow)'].rolling(window=window).mean()
    preProcData['smooth_radius'] = preProcData['circle_radius'].rolling(window=window).mean()  

    preProcData.fillna(method='bfill', inplace=True) # for the beginning
    # .rolling(window=3)
    df = pd.DataFrame(columns=['center_y(ccol)', 'center_x(crow)', 'circle_radius'])

    totalMovement = preProcData['diff_center_x'].sum() + preProcData['diff_center_y'].sum()

    df_list=[]

    if totalMovement < movement_threshold:
        with ThreadPoolExecutor(max_workers=10) as executor:
            files = sorted(os.listdir(output_path))
            
            for i, filename in enumerate(files):
                executor.submit(process_image_low_movement, filename, output_path, preProcData, result_folder, i,df_list)
    else:
        with ThreadPoolExecutor(max_workers=10) as executor:
            files = sorted(os.listdir(output_path))
            for i, filename in enumerate(files):
                executor.submit(process_image_high_movement, filename, output_path, preProcData, result_folder, i,df_list)

    df = pd.concat(df_list, ignore_index=True)
    # print(df)
    df.to_excel(excel_path)


In [7]:
# filter_sizes = [(3, 7)] 
# tests = range(2, 3)

# for filter_size in filter_sizes:
#     for test in tests:
#         input_path = 'ExcelsLarge'
#         input_tiff = f'VideoLarge_object{test}_37_1.xlsx'
#         params = {
#             'r_in': filter_size[0],
#             'r_out': filter_size[1],
#             'ring_index': 1,
#             'allowed_change': 1,
#             'input_path': input_path,
#             'input_tiff': input_tiff,
#             'output_base_path': f'Results_Validation/VideoLarge/cropped_object_{test}'
#         }

#         # try:

#         post_processing_mean(**params)

        # except Exception as e:
        #     print(f"An error occurred: {e}")
        #     print("Params:", params)
        #     continue

In [ ]:
test='2'
input_path='Tiffs_Validation'
input_tiff=f'Test_Large{test}.tif'
# input_tiff=f'Video{test}.tif'
r_in = 3
r_out = 7
ring_index = 1
# brightness=image_background_brightness(get_first_png(input_tiff, input_path))
params={
    'test_number':test,
    'r_in':r_in,
    'r_out':r_out,
    'ring_index':ring_index,
    'allowed_change':1,
    'input_path':input_path,
    'input_tiff':input_tiff,
    'output_base_path':f'Results_Validation/Video_object{test}',
    'output_path':f'Results_Validation/VideoLarge/cropped_object_{test}',
    'result_folder' : f"Results_Validation/{test}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}",
    'excel_input_path':f'ExcelsPrecision/VideoLarge_object{test}_{r_in}{r_out}_{ring_index}_comapprox.xlsx',
    'excel_path' : f'ExcelsPrecision/VideoLarge_object{test}_{r_in}{r_out}_{ring_index}_comapprox_post_processed_mean.xlsx'
}

post_processing_mean(**params)

In [3]:
file_path_1 = 'ExcelsPrecision/VideoLarge_object1_37_1_comapprox_post_processed_mean.xlsx'
file_path_2 = 'ExcelsPrecision/VideoLarge_object2_37_1_comapprox_post_processed_mean.xlsx'

object_1_center = (493, 216)
object_2_center = (332, 730)
half_crop_size = 125  

df1 = pd.read_excel(file_path_1)
df2 = pd.read_excel(file_path_2)

df1['frame'] = df1.index
df2['frame'] = df2.index

df1 = df1.rename(columns={
    'center_x(crow)': '1_x',
    'center_y(ccol)': '1_y',
    'circle_radius': '1_radius'
})
df2 = df2.rename(columns={
    'center_x(crow)': '2_x',
    'center_y(ccol)': '2_y',
    'circle_radius': '2_radius'
})

df1['1_x'] = df1['1_x'] + (object_1_center[0] - half_crop_size)
df1['1_y'] = df1['1_y'] + (object_1_center[1] - half_crop_size)

df2['2_x'] = df2['2_x'] + (object_2_center[0] - half_crop_size)
df2['2_y'] = df2['2_y'] + (object_2_center[1] - half_crop_size)

merged_df = pd.merge(df1, df2, on='frame')

merged_df.to_excel('Consolidated_VideoLarge_info_uncropped_coords.xlsx', index=False)

print("Merged data with uncropped coordinates saved to Consolidated_VideoLarge_info_uncropped_coords.xlsx")

Merged data with uncropped coordinates saved to Consolidated_VideoLarge_info_uncropped_coords.xlsx
